In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

import tensorflow.keras.layers as layers
from sklearn.metrics import confusion_matrix

In [3]:
train_path = 'train.csv'
test_path = 'test.csv'

data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)


for column in data.columns:
    if column == 'Prediction' or column == 'Email No.' : continue
    data[column] = (data[column] - data[column].mean()) / data[column].std()

for column in test_data.columns:
    if column == 'Prediction' or column == 'Email No.' : continue
    test_data[column] = (test_data[column] - test_data[column].mean()) / test_data[column].std()

In [89]:
data = data.sample(frac = 1)
m = data.shape[0]
train_size = int(m * 0.8)
dev_size = m - train_size

y = np.array(data['Prediction'])
x = np.array(data.drop(['Prediction', 'Email No.'], axis=1)).astype('f')

train_x = x[ : train_size , : ]
train_y = y[ : train_size]

dev_x = x[train_size : , : ]
dev_y = y[train_size : ]

# train_data = data[ : train_size]
# dev_data = data[train_size : ]

# train_y = np.array(train_data['Prediction'])
# train_x = np.array(train_data.drop(['Prediction', 'Email No.'], axis=1)).astype('f')

# dev_y = np.array(dev_data['Prediction'])
# dev_x = np.array(dev_data.drop(['Prediction', 'Email No.'], axis=1)).astype('f')

test_x = np.array(test_data.drop(['Email No.'], axis=1).fillna(0)).astype('f')
# test_x = test_data.drop(['Email No.'], axis=1).fillna(0)
# print(test_x.isnull().sum().sum())
# print(dev_x.isnull().sum().sum())

In [90]:
tf.random.set_seed(1)

network1 = keras.Sequential([
    layers.Dense(1024, activation='ReLU', kernel_regularizer=keras.regularizers.L2(1e-2)),
    layers.Dropout(0.15),
    layers.Dense(256, activation='ReLU', kernel_regularizer=keras.regularizers.L2(1e-2)),
    layers.Dropout(0.15),
    layers.Dense(64, activation='ReLU', kernel_regularizer=keras.regularizers.L2(1e-2)),
    layers.Dropout(0.1),
    layers.Dense(8, activation='ReLU', kernel_regularizer=keras.regularizers.L2(1e-2)),
    layers.Dense(1, activation='sigmoid')
])

network1.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

network2 = keras.Sequential([
    layers.Dense(1024, activation='ReLU'),
    layers.Dropout(0.15),
    layers.Dense(2048, activation='ReLU'),
    layers.Dropout(0.3),
    layers.Dense(512, activation='ReLU'),
    layers.Dropout(0.1),
    layers.Dense(32, activation='ReLU'),
    layers.Dense(1, activation='sigmoid')
])

network2.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

In [91]:
network1.fit(train_x, train_y, epochs=30, batch_size=128, verbose=2, validation_data=[dev_x, dev_y])
network2.fit(train_x, train_y, epochs=30, batch_size=128, verbose=2, validation_data=[dev_x, dev_y])

Epoch 1/30
31/31 - 2s - loss: 9.0393 - accuracy: 0.8733 - val_loss: 2.3253 - val_accuracy: 0.9643 - 2s/epoch - 63ms/step
Epoch 2/30
31/31 - 1s - loss: 1.1852 - accuracy: 0.9607 - val_loss: 0.5949 - val_accuracy: 0.9429 - 1s/epoch - 43ms/step
Epoch 3/30
31/31 - 1s - loss: 0.4139 - accuracy: 0.9681 - val_loss: 0.2991 - val_accuracy: 0.9531 - 1s/epoch - 41ms/step
Epoch 4/30
31/31 - 1s - loss: 0.2449 - accuracy: 0.9686 - val_loss: 0.2117 - val_accuracy: 0.9673 - 1s/epoch - 42ms/step
Epoch 5/30
31/31 - 1s - loss: 0.1826 - accuracy: 0.9696 - val_loss: 0.1676 - val_accuracy: 0.9592 - 1s/epoch - 45ms/step
Epoch 6/30
31/31 - 2s - loss: 0.1584 - accuracy: 0.9676 - val_loss: 0.1423 - val_accuracy: 0.9673 - 2s/epoch - 59ms/step
Epoch 7/30
31/31 - 1s - loss: 0.1279 - accuracy: 0.9788 - val_loss: 0.1240 - val_accuracy: 0.9796 - 1s/epoch - 45ms/step
Epoch 8/30
31/31 - 1s - loss: 0.1227 - accuracy: 0.9724 - val_loss: 0.1228 - val_accuracy: 0.9796 - 1s/epoch - 39ms/step
Epoch 9/30
31/31 - 1s - loss: 0.

In [85]:
predictions = network1.predict(dev_x) + network2
predictions1 = np.where(predictions > 0.5, 1, 0).astype(int)

matrix = confusion_matrix(dev_y, predictions)
print(matrix.diagonal() / matrix.sum(axis=1))
predictions.sum() / predictions.shape[0]

[0.9829303  0.97833935]


0.28877551020408165

In [65]:
test = pd.read_csv('sample_submission.csv')
test_pred = network1.predict(test_x)
test_pred = np.where(test_pred > 0.5, 1, 0).astype(int).flatten()
test['Prediction'] = pd.Series(test_pred)
test.to_csv('submission.csv', index=False)